In [ ]:
import numpy as np

NUM_SLOTS = 48  # 12 hours * 4 slots per hour

def hhmm_to_slot(hhmm: str) -> int:
    t = int(hhmm)
    h = t // 100
    m = t % 100
    slot = (h - 10) * 4 + (m // 15)
    return max(0, min(NUM_SLOTS - 1, slot))

def parse_availability(avail_str: str) -> list[int]:
    slots = [0] * NUM_SLOTS
    intervals = [s.strip() for s in avail_str.split(",")]
    for interval in intervals:
        if not interval:
            continue
        start, end = interval.split("-")
        s, e = hhmm_to_slot(start), hhmm_to_slot(end)
        slots[s] = 1
        slots[min(e,NUM_SLOTS)] = -1
    return slots

# Input availability strings
input_avail = [
    '1000-1200','2000-2200','1300-1430,2030-2200','1300-1430,2030-2200',
    '1300-1430,2030-2200,1000-1130','1000-1600','1000-1600','1030-1900',
    '1030-1900','1030-1900','1030-1900','1030-1900','1100-2200','1100-2200',
    '1100-2200','1200-2200','1200-2200','1145-1830','1145-2200','1200-2200',
    '1145-2200','1145-2200','1230-1400','1130-1300','1300-1430','1230-1630',
    '1600-1830','1600-1830','1400-1830','1400-1830','1000-1200','2000-2200',
    '1800-2030','1700-2200'
]

# Convert to matrix
schedule_matrix = np.array([parse_availability(av) for av in input_avail])

print(schedule_matrix.shape)  # (34, 48) -> 34 officers × 48 slots
np.set_printoptions(threshold=np.inf)

print(schedule_matrix)


In [ ]:
officer_in_out = schedule_matrix.sum(axis=0).tolist()
print(officer_in_out)
print(len(officer_in_out))

In [ ]:
NUM_SLOTS = 48  # 12 hours * 4 slots per hour

def hhmm_to_slot(hhmm: str) -> int:
    t = int(hhmm)
    h = t // 100
    m = t % 100
    slot = (h - 10) * 4 + (m // 15)
    return max(0, min(NUM_SLOTS - 1, slot))

def parse_availability(avail_str: str) -> list[int]:
    slots = [0] * NUM_SLOTS
    intervals = [s.strip() for s in avail_str.split(",")]
    for interval in intervals:
        if not interval:
            continue
        start, end = interval.split("-")
        s, e = hhmm_to_slot(start), hhmm_to_slot(end)
        slots[s] = 1
        slots[min(e,NUM_SLOTS)] = -1
    return slots

def slots_to_intervals(slots: list[int]) -> list[tuple[int, int]]:
    intervals = []
    in_interval = False
    start = 0
    for i, v in enumerate(slots):
        if v == 1 and not in_interval:
            start = i
            in_interval = True
        elif v == -1 and in_interval:
            intervals.append((start, i - 1))
            in_interval = False
    if in_interval:
        intervals.append((start, NUM_SLOTS - 1))
    return intervals

# Input availability strings
input_avail = [
    '1000-1200','2000-2200','1300-1430,2030-2200','1300-1430,2030-2200',
    '1300-1430,2030-2200,1000-1130','1000-1600','1000-1600','1030-1900',
    '1030-1900','1030-1900','1030-1900','1030-1900','1100-2200','1100-2200',
    '1100-2200','1200-2200','1200-2200','1145-1830','1145-2200','1200-2200',
    '1145-2200','1145-2200','1230-1400','1130-1300','1300-1430','1230-1630',
    '1600-1830','1600-1830','1400-1830','1400-1830','1000-1200','2000-2200',
    '1800-2030','1700-2200'
]

def init_break_planner(input_avail):
    # Build dict {officer_id: [(start_slot,end_slot), ...]}
    officer_intervals = {
        officer_id + 1: slots_to_intervals(parse_availability(av))
        for officer_id, av in enumerate(input_avail)
    }

    schedule_matrix = np.array([parse_availability(av) for av in input_avail])

    print(officer_intervals)
    return officer_intervals, schedule_matrix


In [ ]:
from collections import defaultdict

# officer_intervals is from the previous step
inverted_dict = defaultdict(list)

for officer_id, intervals in officer_intervals.items():
    for interval in intervals:
        inverted_dict[interval].append(officer_id)

# Convert back to a normal dict if desired
inverted_dict = dict(inverted_dict)

print(inverted_dict)
# key is sos start, end 
# value is list of officer_id


In [ ]:
next_open_counter = [40,30,20,10,39,29,19,9,38,28,18,8,37,27,17,7,36,26,16,6,35,25,15,5,
                     34,24,14,4,33,23,13,3,32,22,12,2,31,21,11,1]
num_officers = len(input_avail)  # change this to however many officers you have

officer_dict = {i: [] for i in range(1, num_officers + 1)}

print(officer_dict)

In [ ]:
def calculate_total_break(officer_intervals):
    officers_break_quota = {}
    for officer in officer_intervals:
        total_break = []
        for each_interval in officer_intervals[officer]:
            this_break = 0
            s,e = each_interval
            if e-s >= 36:
                total_break = [2,3,3]
                break
            elif e-s >= 20:
                this_break = 3
            elif e-s >= 10:
                this_break = 2
        if this_break > 0:
            total_break.append(this_break)
        officers_break_quota[officer] = total_break
    return officers_break_quota


#officers_break_quota = calculate_total_break(officer_intervals)
#print(officers_break_quota)

In [ ]:

def check_break_eligibility(t, officers_break_quota, officer_intervals):
    #if 4 < t <= NUM_SLOTS-4 : 
    officers_upcoming_break = {k: v[0] for k, v in officers_break_quota.items() if v} # return officer id and length of upcoming break
    eligible_officers_break = { # no officers should go for break and at the start (1st h) and end (last h) of each sos period
    officer_id: length
    for officer_id, length in officers_upcoming_break.items()
    if any(start + 4 < t <= end-4-length for start, end in officer_intervals.get(officer_id, []))
    }
    return eligible_officers_break #return dict {officer_id:break_time}

# for t in range(NUM_SLOTS):
#     print('t',t)
#     eligible_officers_break = check_break_eligibility(t, officers_break_quota,officer_intervals)
#     print(eligible_officers_break)

#eligible_officers_break = check_break_eligibility(t, officers_break_quota,officer_intervals)

In [ ]:
eligible_officers_break

In [ ]:
#break_schedule = {officer_id: [] for officer_id in officer_intervals}
def update_break_schedule(n_officers, eligible_officers_break, t, officers_break_quota, break_schedule, officer_in_out): # n_officers represents officer count to reach equilibrium
    # Populate based on length
    officer_count = 0
    common_officers = list(break_schedule.keys() & eligible_officers_break.keys())
    if len(common_officers) == 0:
        return break_schedule, officers_break_quota, officer_in_out
    else:
        for common_officers in eligible_officers_break:
            length = eligible_officers_break[common_officers]
            break_schedule[common_officers].append([t + j for j in range(length)]) # add break schedule
            officers_break_quota[common_officers].pop(0) # update officer remaining break left
            officer_count += 1
            officer_in_out[t] -= 1
            officer_in_out[t+length] += 1
            if officer_count == n_officers:
                break
        return break_schedule, officers_break_quota,officer_in_out



In [ ]:
officer_in_out[3]

In [ ]:
schedule_with_breaks = schedule_matrix.copy()

# Make the new matrix object type so we can store '#'
schedule_with_breaks = schedule_with_breaks.astype(object)


In [ ]:
schedule_matrix


In [ ]:
schedule_with_breaks

In [ ]:
# Mark the breaks
for officer_id, break_slots in flat_dict.items():
    if break_slots:
        row_idx = officer_id - 1
        for col_idx in break_slots:
            schedule_with_breaks[row_idx, col_idx] = '#'  # mark break

In [ ]:
officer_intervals, schedule_matrix = init_break_planner(input_avail)
officers_break_quota = calculate_total_break(officer_intervals)

def main_loop(schedule_matrix, officer_intervals, officers_break_quota):
    break_schedule = {officer_id: [] for officer_id in officer_intervals}
    officer_in_out = schedule_matrix.sum(axis=0).tolist()
    for t in range(NUM_SLOTS):
        officer_in = np.where(schedule_matrix[:, t] == 1)[0]
        officer_out = np.where(schedule_matrix[:, t] == -1)[0]
        print('t:', t)
        if officer_in.size != 0:
            print(officer_in)
        if officer_out.size != 0:
            print(officer_out)
        if officer_in_out[t] > 0:
            if t < 4: # too early to go for break, incoming officers will open new counters
                print ('open new counter')
            else:
                print ('net in,', officer_in_out[t], 'officers can go for break now')
                eligible_officers_break = check_break_eligibility(t, officers_break_quota,officer_intervals)
                '______________'
                print(eligible_officers_break)
                print(officer_in_out[t])
                print ('eligible',eligible_officers_break)
                print (t)
                updated_break_schedule = update_break_schedule(officer_in_out[t], eligible_officers_break, t, officers_break_quota, break_schedule, officer_in_out)
                break_schedule, officers_break_quota, officer_in_out = updated_break_schedule
                print ('officer_in_out:', officer_in_out)
        elif officer_in_out[t] == 0:
            print ('in equilibrium, no break now') #allocating any break now will disort the equilibrium
        else:
            if t == NUM_SLOTS-1:
                print('shift ended!')
            else:
                print ('net out,', officer_in_out[t]*(-1), 'officers to come back from break now') # to take over counters

    return updated_break_schedule

updated_break_schedule = main_loop(schedule_matrix, officer_intervals, officers_break_quota) 


In [ ]:
flat_dict = {
    k: [item for sublist in v for item in sublist] if v else []
    for k, v in updated_break_schedule[0].items()}

In [ ]:
schedule_matrix

In [ ]:
print(updated_break_schedule[0])
updated_break_final = [item for sublist in flat for item in sublist]

In [ ]:
# Sort by tuple naturally (first element, then second element)
sorted_inverted = dict(sorted(inverted_dict.items(), key=lambda x: (x[0][0], x[0][1])))

print(sorted_inverted)


In [ ]:
empty_counter = [1,2,3,4,5,6,7,8,9,10]

In [ ]:
for sos_period in sorted_inverted:
    if sos_period[0] == 0:
        for s in sorted_inverted[sos_period]:
            start, end = sos_period
            schedule_matrix[s, start:end+1] = empty_counter[-1]

            schedule_matrix[s][0] = empty_counter[-1]
            empty_counter.pop()
            print(empty_counter)

            print(schedule_matrix)

In [ ]:
input_avail = [
    '1000-1200','2000-2200','1300-1430,2030-2200','1300-1430,2030-2200',
    '1300-1430,2030-2200,1000-1130','1000-1600','1000-1600','1030-1900',
    '1030-1900','1030-1900','1030-1900','1030-1900','1100-2200','1100-2200',
    '1100-2200','1200-2200','1200-2200','1145-1830','1145-2200','1200-2200',
    '1145-2200','1145-2200','1230-1400','1130-1300','1300-1430','1230-1630',
    '1600-1830','1600-1830','1400-1830','1400-1830','1000-1200','2000-2200',
    '1800-2030','1700-2200'
]

In [108]:
NUM_SLOTS = 48  # 12h shift, 15 min per slot
MAX_CONSECUTIVE = 12

def calculate_total_break2(officer_intervals):
    officers_break_quota = {}
    for officer in officer_intervals:
        total_break = 0
        for each_interval in officer_intervals[officer]:
            s, e = each_interval
            if e - s >= 36:   # >= 9 hours
                total_break = 8
                break
            elif e - s >= 20: # >= 5 hours
                total_break = max(total_break, 3)
            elif e - s >= 10: # >= 2.5 hours
                total_break = max(total_break, 2)
        officers_break_quota[officer] = total_break
    return officers_break_quota


def build_model_inputs(input_avail: list[str], counter_count = 40):
    officers = [f"O{i+1}" for i in range(len(input_avail))]
    counters = [f"C{str(i+1)}" for i in range(counter_count)]  # 40 counters
    
    # Step 1: Parse availability into slot ranges per officer
    officer_intervals = {}
    for i, avail_str in enumerate(input_avail):
        intervals = [s.strip() for s in avail_str.split(",")]
        ranges = []
        for interval in intervals:
            if not interval:
                continue
            start, end = interval.split("-")
            s, e = hhmm_to_slot(start), hhmm_to_slot(end)
            ranges.append((s, e))
        officer_intervals[officers[i]] = ranges
    
    # Step 2: Flatten ranges into all slots available
    availability = {}
    for officer, ranges in officer_intervals.items():
        slots = set()
        for s, e in ranges:
            slots.update(range(s, e))
        availability[officer] = sorted(list(slots))
    
    # Step 3: Break requirements
    break_requirements = calculate_total_break2(officer_intervals)
    
    return officers, counters, availability, break_requirements

officers, counters, availability, break_requirements = build_model_inputs(input_avail)
print (officers)
print(counters)
print(availability)
print(break_requirements)


['O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13', 'O14', 'O15', 'O16', 'O17', 'O18', 'O19', 'O20', 'O21', 'O22', 'O23', 'O24', 'O25', 'O26', 'O27', 'O28', 'O29', 'O30', 'O31', 'O32', 'O33', 'O34']
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40']
{'O1': [0, 1, 2, 3, 4, 5, 6, 7], 'O2': [40, 41, 42, 43, 44, 45, 46], 'O3': [12, 13, 14, 15, 16, 17, 42, 43, 44, 45, 46], 'O4': [12, 13, 14, 15, 16, 17, 42, 43, 44, 45, 46], 'O5': [0, 1, 2, 3, 4, 5, 12, 13, 14, 15, 16, 17, 42, 43, 44, 45, 46], 'O6': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], 'O7': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], 'O8': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 

In [ ]:
from ortools.sat.python import cp_model

# ----------------------------
# Setup
# ----------------------------
officers = ["A", "B", "C"]
counters = ["C1", "C2"]
num_slots = 48  # Example: 48 slots for 12h, each slot 15 min

# Officer availability (which slots they can work)
availability = {
    "A": range(3, 6),    # A works slot 3–5 only
    "B": range(5, 13),   # B works slot 5–12
    "C": range(0, 13),   # C works whole shift
}

# Minimum breaks required per officer
break_requirements = {
    "A": 1,
    "B": 2,
    "C": 3
}

# Max consecutive work slots (10 = 2h30 if each slot is 15min)
MAX_CONSECUTIVE = 10

officers, counters, availability, break_requirements = build_model_inputs(input_avail)


# ----------------------------
# Model
# ----------------------------
model = cp_model.CpModel()

# Variables: assign[o][c][t] = 1 if officer o works at counter c in slot t
assign = {}
break_var = {}
for o in officers:
    for c in counters:
        for t in range(num_slots):
            assign[(o, c, t)] = model.NewBoolVar(f"{o}_{c}_{t}")
    for t in range(num_slots):
        break_var[(o, t)] = model.NewBoolVar(f"{o}_break_{t}")


# ----------------------------
# Constraints
# ----------------------------

# 1) At most 1 officer per counter per slot
for c in counters:
    for t in range(num_slots):
        model.Add(sum(assign[(o, c, t)] for o in officers) <= 1)

# 2) Each officer must either work at 1 counter OR take break in each slot
for o in officers:
    for t in range(num_slots):
        model.Add(sum(assign[(o, c, t)] for c in counters) + break_var[(o, t)] == 1)

# 3) Max consecutive work duration (<= MAX_CONSECUTIVE slots)
for o in officers:
    for start in range(num_slots - MAX_CONSECUTIVE + 1):
        model.Add(
            sum(assign[(o, c, t)] for c in counters for t in range(start, start + MAX_CONSECUTIVE))
            <= MAX_CONSECUTIVE
        )

# 4) Enforce officer availability
for o in officers:
    for t in range(num_slots):
        if t not in availability[o]:
            model.Add(break_var[(o, t)] == 1)  # must be on break outside availability
            for c in counters:
                model.Add(assign[(o, c, t)] == 0)

# 5) Break requirements (support multiple blocks of specific lengths)
    for o, reqs in break_requirements.items():
        for i, length in enumerate(reqs):
            # Create an interval variable for each break block
            start = model.NewIntVar(0, num_slots - length, f"{o}_break{i}_start")
            interval = model.NewIntervalVar(start, length, start + length, f"{o}_break{i}_int")

            # Enforce that those slots are breaks
            for t in range(num_slots - length + 1):
                cond = model.NewBoolVar(f"{o}_break{i}_start_{t}")
                model.Add(start == t).OnlyEnforceIf(cond)
                for k in range(length):
                    model.Add(break_var[(o, t + k)] == 1).OnlyEnforceIf(cond)


# ----------------------------
# Objective: maximize counter coverage
# ----------------------------
model.Maximize(sum(assign[(o, c, t)] for o in officers for c in counters for t in range(num_slots)))

# ----------------------------
# Solve
# ----------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 20
status = solver.Solve(model)

# 

In [ ]:
# Print solution
# ----------------------------
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("Solution found!\n")
    # for t in range(num_slots):
    #     print(f"Slot {t}:")
    #     for c in counters:
    #         assigned = [o for o in officers if solver.Value(assign[(o, c, t)])]
    #         if assigned:
    #             print(f"  {c}: Officer {assigned[0]}")
    #         else:
    #             print(f"  {c}: EMPTY")
    #     for o in officers:
    #         if solver.Value(break_var[(o, t)]):
    #             print(f"  Officer {o} on BREAK")
    #     print()
    # Print officer rosters
    print("\n=== Officer Rosters ===")
    for o in officers:
        roster = []
        for t in range(num_slots):
            if solver.Value(break_var[(o, t)]) == 1:
                roster.append("#")
            else:
                for c in counters:
                    if solver.Value(assign[(o, c, t)]) == 1:
                        roster.append(c)
        print(f"Officer {o}: {roster}")

    # Print counter manning
    print("\n=== Counter Manning ===")
    for c in counters:
        manning = [sum(solver.Value(assign[(o, c, t)]) for o in officers) for t in range(num_slots)]
        print(f"{c}: {manning}")
else:
    print("No solution found")

In [ ]:
from ortools.sat.python import cp_model

# ----------------------------
# Setup
# ----------------------------
officers = ["A", "B", "C"]
counters = ["C1", "C2"]
num_slots = 16
MAX_CONSECUTIVE = 5

# Example availability
availability = {
    "A": range(0, num_slots),
    "B": range(0, num_slots),
    "C": range(0, num_slots),
}

# Break requirements: list of required block sizes per officer
break_requirements = {
    "A": [1, 2],      # needs one 1-slot break, one 2-slot break
    "B": [2, 3],      # needs one 2-slot break, one 3-slot break
    "C": [2, 3, 3]    # needs three breaks with lengths 2, 3, and 3
}

# ----------------------------
# Model
# ----------------------------
model = cp_model.CpModel()

# Decision variables
assign = {}
brk = {}
for o in officers:
    for c in counters:
        for t in range(num_slots):
            assign[(o, c, t)] = model.NewBoolVar(f"{o}_{c}_{t}")
    for t in range(num_slots):
        brk[(o, t)] = model.NewBoolVar(f"{o}_break_{t}")

# Each officer per slot: work at one counter OR take break OR idle
for o in officers:
    for t in range(num_slots):
        model.Add(
            sum(assign[(o, c, t)] for c in counters) + brk[(o, t)] <= 1
        )

# Exactly 1 officer per counter per slot
for c in counters:
    for t in range(num_slots):
        model.Add(
            sum(assign[(o, c, t)] for o in officers) == 1
        )

# Max consecutive work constraint
for o in officers:
    for start in range(num_slots - MAX_CONSECUTIVE):
        model.Add(
            sum(assign[(o, c, t)] for c in counters for t in range(start, start + MAX_CONSECUTIVE + 1))
            <= MAX_CONSECUTIVE
        )

# Break block requirements
break_start = {}
for o in officers:
    for i, L in enumerate(break_requirements[o]):
        for t in range(num_slots - L + 1):
            break_start[(o, i, t)] = model.NewBoolVar(f"{o}_break{i}_start{t}")

        # Each break block must start exactly once
        model.Add(
            sum(break_start[(o, i, t)] for t in range(num_slots - L + 1)) == 1
        )

        # Expand block into actual break slots
        for t in range(num_slots - L + 1):
            for k in range(L):
                model.AddImplication(break_start[(o, i, t)], brk[(o, t+k)])

# ----------------------------
# Objective: (minimize idle or just satisfy constraints)
# ----------------------------
model.Maximize(1)  # dummy objective

# ----------------------------
# Solve
# ----------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 20
status = solver.Solve(model)

# ----------------------------
# Output
# ----------------------------
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("\n=== Officer Rosters ===")
    for o in officers:
        roster = []
        for t in range(num_slots):
            if solver.Value(brk[(o, t)]):
                roster.append("# ")
            else:
                found = False
                for c in counters:
                    if solver.Value(assign[(o, c, t)]):
                        roster.append(c)
                        found = True
                        break
                if not found:
                    roster.append("> ")
        print(f"Officer {o}: {roster}")

    print("\n=== Counter Manning ===")
    for c in counters:
        manning = [sum(solver.Value(assign[(o, c, t)]) for o in officers) for t in range(num_slots)]
        print(f"{c}: {manning}")

else:
    print("No feasible solution found.")


In [ ]:
from itertools import product

# Input
officers = ["A", "B", "C"]
counters = ["C1", "C2"]
slots = 16
max_consecutive = 5

break_requirements = {
    "A": [1, 2],
    "B": [2, 3],
    "C": [2, 3, 3]
}

# Initialize officer schedules with None
schedule = {off: ['#'] * slots for off in officers}

def valid_schedule(officer_schedule, max_consec):
    """Check if officer schedule does not exceed max consecutive work."""
    count = 0
    for slot in officer_schedule:
        if slot != '#':
            count += 1
            if count > max_consec:
                return False
        else:
            count = 0
    return True

def assign_breaks(off, req_blocks):
    """Return all possible ways to assign breaks as per block requirements."""
    from itertools import combinations
    
    free_slots = list(range(slots))
    possible_positions = []
    
    def backtrack(block_idx, current_schedule, used_slots):
        if block_idx == len(req_blocks):
            possible_positions.append(current_schedule[:])
            return
        block_len = req_blocks[block_idx]
        # Find start positions where block can fit
        for start in range(slots - block_len + 1):
            block_range = set(range(start, start + block_len))
            if not block_range & used_slots:
                # Place break
                for i in range(start, start + block_len):
                    current_schedule[i] = '#'
                backtrack(block_idx + 1, current_schedule, used_slots | block_range)
                # Remove break to backtrack
                for i in range(start, start + block_len):
                    current_schedule[i] = None
                    
    backtrack(0, [None] * slots, set())
    return possible_positions

# Generate all possible break schedules per officer
officer_break_options = {}
for off in officers:
    officer_break_options[off] = assign_breaks(off, break_requirements[off])

# Try combinations
from itertools import product

for combo in product(*officer_break_options.values()):
    temp_schedule = {off: list(s) for off, s in zip(officers, combo)}
    # Fill remaining slots with counters trying to maximize consecutive coverage
    counter_idx = 0
    for off in officers:
        for i in range(slots):
            if temp_schedule[off][i] is None:
                temp_schedule[off][i] = counters[counter_idx % len(counters)]
        # Check max consecutive constraint
        if not valid_schedule(temp_schedule[off], max_consecutive):
            break
    else:
        # Found feasible schedule
        schedule = temp_schedule
        break

# Compute counter manning
counter_manning = {c: [0]*slots for c in counters}
for off in officers:
    for i, val in enumerate(schedule[off]):
        if val in counters:
            counter_manning[val][i] += 1

# Print
print("=== Officer Rosters ===")
for off in officers:
    print(f"{off}: {schedule[off]}")

print("\n=== Counter Manning ===")
for c in counters:
    print(f"{c}: {counter_manning[c]}")


In [ ]:
from ortools.sat.python import cp_model

# === Parameters ===
officers = ["A", "B", "C"]
counters = ["C1", "C2", "C3"]
slots = list(range(15))

# Minimum total break slots per officer
min_total_breaks = {
    "A": 4,
    "B": 3,
    "C": 5
}

# Allowed break block lengths
break_lengths = [2, 3]

model = cp_model.CpModel()

# === Decision Variables ===
officer_slot = {}
for o in officers:
    for s in slots:
        officer_slot[o, s] = model.NewIntVar(-1, len(counters)-1, f"officer_{o}_slot_{s}")

# === Constraints ===
for o in officers:
    # Boolean indicator: 1 if on break, 0 if working
    is_break = [model.NewBoolVar(f"break_{o}_{s}") for s in slots]
    for s in slots:
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_break[s])
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_break[s].Not())
    
    # Minimum total breaks
    model.Add(sum(is_break) >= min_total_breaks[o])
    
    # Break blocks with no consecutive merges
    block_starts = []
    s = 0
    while s < len(slots):
        for length in break_lengths:
            if s + length <= len(slots):
                block = [is_break[s + i] for i in range(length)]
                block_start = model.NewBoolVar(f"break_block_{o}_{s}_len{length}")
                block_starts.append((s, length, block_start))
                
                model.Add(sum(block) == length).OnlyEnforceIf(block_start)
                model.Add(sum(block) != length).OnlyEnforceIf(block_start.Not())
                
                # Ensure next slot after block is working
                if s + length < len(slots):
                    model.Add(is_break[s + length] == 0).OnlyEnforceIf(block_start)
        s += 1
    
    # Ensure total break slots from selected blocks >= min_total_breaks
    block_lengths_list = [length for (_, length, _) in block_starts]
    block_vars_list = [var for (_, _, var) in block_starts]
    model.Add(sum([l * v for l, v in zip(block_lengths_list, block_vars_list)]) >= min_total_breaks[o])
    
    # === Stay at same counter until break ===
    for s in range(1, len(slots)):
        # Only enforce if both slots are working
        prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
        model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
        model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())
        
        curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())
        
        both_working = model.NewBoolVar(f"{o}_both_working_{s}")
        model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
        model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())
        
        # If both working, enforce same counter
        model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(both_working)

# === Objective: maximize total officers working ===
working_slots = []
for o in officers:
    for s in slots:
        is_working = model.NewBoolVar(f"working_{o}_{s}")
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_working)
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_working.Not())
        working_slots.append(is_working)

model.Maximize(sum(working_slots))

# === Constraint: max 1 officer per counter per slot ===
for c_index, c in enumerate(counters):
    for s in slots:
        working_at_counter = []
        for o in officers:
            is_at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
            model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(is_at_c)
            model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(is_at_c.Not())
            working_at_counter.append(is_at_c)
        model.Add(sum(working_at_counter) <= 1)

# === Solve ===
solver = cp_model.CpSolver()
status = solver.Solve(model)

# === Print schedules and counter manning ===
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("=== Officer Schedules ===")
    for o in officers:
        schedule = []
        for s in slots:
            val = solver.Value(officer_slot[o, s])
            schedule.append('#' if val == -1 else counters[val])
        print(f"{o}: {schedule}")
    
    # Counter Manning
    counter_manning = {c: [] for c in counters}
    for s in slots:
        manning = {c: 0 for c in counters}
        for o in officers:
            val = solver.Value(officer_slot[o, s])
            if val != -1:
                manning[counters[val]] += 1
        for c in counters:
            counter_manning[c].append(manning[c])
    
    print("\n=== Counter Manning ===")
    for c in counters:
        print(f"{c}: {counter_manning[c]}")
else:
    print("No feasible solution found.")


In [ ]:
from ortools.sat.python import cp_model

# === Parameters ===
officers = ["A", "B", "C"]
counters = ["C1", "C2", "C3"]
slots = list(range(15))

# Officer availability (start_slot, end_slot)
sos_availability = {
    "A": (1, 12),
    "B": (0, 12),
    "C": (0, 12)
}

# Minimum total break slots per officer
min_total_breaks = {
    "A": 4,
    "B": 3,
    "C": 5
}

# Allowed break block lengths
break_lengths = [2, 3]

model = cp_model.CpModel()

# === Decision Variables ===
officer_slot = {}
for o in officers:
    for s in slots:
        officer_slot[o, s] = model.NewIntVar(-1, len(counters)-1, f"officer_{o}_slot_{s}")

# === Constraints ===
for o in officers:
    start_avail, end_avail = sos_availability[o]
    
    # Boolean indicator: 1 if on break
    is_break = [model.NewBoolVar(f"break_{o}_{s}") for s in slots]
    for s in slots:
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_break[s])
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_break[s].Not())
    
    # Minimum total breaks
    model.Add(sum(is_break) >= min_total_breaks[o])
    
    # Break blocks with no consecutive merges
    block_starts = []
    s = 0
    while s < len(slots):
        for length in break_lengths:
            if s + length <= len(slots):
                block = [is_break[s + i] for i in range(length)]
                block_start = model.NewBoolVar(f"break_block_{o}_{s}_len{length}")
                block_starts.append((s, length, block_start))
                
                model.Add(sum(block) == length).OnlyEnforceIf(block_start)
                model.Add(sum(block) != length).OnlyEnforceIf(block_start.Not())
                
                # Ensure next slot after block is working
                if s + length < len(slots):
                    model.Add(is_break[s + length] == 0).OnlyEnforceIf(block_start)
        s += 1
    
    # Ensure total break slots from selected blocks >= min_total_breaks
    block_lengths_list = [length for (_, length, _) in block_starts]
    block_vars_list = [var for (_, _, var) in block_starts]
    model.Add(sum([l * v for l, v in zip(block_lengths_list, block_vars_list)]) >= min_total_breaks[o])
    
    # Stay at same counter until break
    for s in range(1, len(slots)):
        prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
        model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
        model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())
        
        curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())
        
        both_working = model.NewBoolVar(f"{o}_both_working_{s}")
        model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
        model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())
        
        model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(both_working)

# === Objective: working slots + penalties ===
working_slots = []
counter_change_penalties = []
break_timing_penalties = []

for o in officers:
    start_avail, end_avail = sos_availability[o]
    for s in slots:
        # Working slot
        is_working = model.NewBoolVar(f"working_{o}_{s}")
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_working)
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_working.Not())
        working_slots.append(is_working)
        
        # Break timing penalty: if break in first 4 or last 4 slots of availability
        if s - start_avail < 4 or end_avail - s <= 4:
            break_penalty = model.NewBoolVar(f"break_penalty_{o}_{s}")
            model.Add(is_break[s] == 1).OnlyEnforceIf(break_penalty)
            model.Add(is_break[s] == 0).OnlyEnforceIf(break_penalty.Not())
            break_timing_penalties.append(break_penalty)
            
    # Counter change penalty
    # Counter change penalty
for o in officers:
    for s in range(1, len(slots)):
        # Indicator: changed counters between consecutive slots
        changed = model.NewBoolVar(f"{o}_changed_{s}")
        model.Add(officer_slot[o, s] != officer_slot[o, s-1]).OnlyEnforceIf(changed)
        model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(changed.Not())
        
        # Only penalize if both slots are working
        prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
        model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
        model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())
        curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())
        
        both_working = model.NewBoolVar(f"{o}_both_working_change_{s}")
        model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
        model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())
        
        # New Boolean for penalty: both working AND changed
        penalty_if_working = model.NewBoolVar(f"penalty_if_working_{o}_{s}")
        # If changed AND both_working -> penalty_if_working = 1
        model.AddBoolAnd([changed, both_working]).OnlyEnforceIf(penalty_if_working)
        # Otherwise penalty_if_working = 0
        model.AddBoolOr([changed.Not(), both_working.Not()]).OnlyEnforceIf(penalty_if_working.Not())
        
        counter_change_penalties.append(penalty_if_working)


# Max 1 officer per counter
for c_index, c in enumerate(counters):
    for s in slots:
        working_at_counter = []
        for o in officers:
            is_at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
            model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(is_at_c)
            model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(is_at_c.Not())
            working_at_counter.append(is_at_c)
        model.Add(sum(working_at_counter) <= 1)

# === Combined Objective ===
# Maximize working slots, minimize counter changes and bad break timing
# Small weights for penalties
model.Maximize(
    sum(working_slots)
    - 0.5 * sum(counter_change_penalties)
    - 2.0 * sum(break_timing_penalties)
)

# === Solve ===
solver = cp_model.CpSolver()
status = solver.Solve(model)

# === Print schedules and counter manning ===
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("=== Officer Schedules ===")
    for o in officers:
        schedule = []
        for s in slots:
            val = solver.Value(officer_slot[o, s])
            schedule.append('#' if val == -1 else counters[val])
        print(f"{o}: {schedule}")
    
    # Counter Manning
    counter_manning = {c: [] for c in counters}
    for s in slots:
        manning = {c: 0 for c in counters}
        for o in officers:
            val = solver.Value(officer_slot[o, s])
            if val != -1:
                manning[counters[val]] += 1
        for c in counters:
            counter_manning[c].append(manning[c])
    
    print("\n=== Counter Manning ===")
    for c in counters:
        print(f"{c}: {counter_manning[c]}")
else:
    print("No feasible solution found.")


In [ ]:
from ortools.sat.python import cp_model

# === Parameters ===
officers = ["A", "B", "C"]
counters = ["C1", "C2", "C3"]
slots = list(range(15))

# Minimum total break slots per officer
min_total_breaks = {
    "A": 4,
    "B": 3,
    "C": 5
}

# Allowed break block lengths
break_lengths = [2, 3]

model = cp_model.CpModel()

# === Decision Variables ===
officer_slot = {}
for o in officers:
    for s in slots:
        officer_slot[o, s] = model.NewIntVar(-1, len(counters)-1, f"officer_{o}_slot_{s}")

# === Constraints ===
for o in officers:
    # Boolean indicator: 1 if on break, 0 if working
    is_break = [model.NewBoolVar(f"break_{o}_{s}") for s in slots]
    for s in slots:
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_break[s])
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_break[s].Not())
    
    # Minimum total breaks
    model.Add(sum(is_break) >= min_total_breaks[o])
    
    # Break blocks with no consecutive merges
    block_starts = []
    s = 0
    while s < len(slots):
        for length in break_lengths:
            if s + length <= len(slots):
                block = [is_break[s + i] for i in range(length)]
                block_start = model.NewBoolVar(f"break_block_{o}_{s}_len{length}")
                block_starts.append((s, length, block_start))
                
                model.Add(sum(block) == length).OnlyEnforceIf(block_start)
                model.Add(sum(block) != length).OnlyEnforceIf(block_start.Not())
                
                # Ensure next slot after block is working
                if s + length < len(slots):
                    model.Add(is_break[s + length] == 0).OnlyEnforceIf(block_start)
        s += 1
    
    # Ensure total break slots from selected blocks >= min_total_breaks
    block_lengths_list = [length for (_, length, _) in block_starts]
    block_vars_list = [var for (_, _, var) in block_starts]
    model.Add(sum([l * v for l, v in zip(block_lengths_list, block_vars_list)]) >= min_total_breaks[o])
    
    # === Stay at same counter until break ===
    for s in range(1, len(slots)):
        # Only enforce if both slots are working
        prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
        model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
        model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())
        
        curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())
        
        both_working = model.NewBoolVar(f"{o}_both_working_{s}")
        model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
        model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())
        
        # If both working, enforce same counter
        model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(both_working)

# === Objective: maximize total officers working ===
working_slots = []
for o in officers:
    for s in slots:
        is_working = model.NewBoolVar(f"working_{o}_{s}")
        model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_working)
        model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_working.Not())
        working_slots.append(is_working)

model.Maximize(sum(working_slots))

# === Constraint: max 1 officer per counter per slot ===
for c_index, c in enumerate(counters):
    for s in slots:
        working_at_counter = []
        for o in officers:
            is_at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
            model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(is_at_c)
            model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(is_at_c.Not())
            working_at_counter.append(is_at_c)
        model.Add(sum(working_at_counter) <= 1)

# === Solve ===
solver = cp_model.CpSolver()
status = solver.Solve(model)

# === Print schedules and counter manning ===
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("=== Officer Schedules ===")
    for o in officers:
        schedule = []
        for s in slots:
            val = solver.Value(officer_slot[o, s])
            schedule.append('#' if val == -1 else counters[val])
        print(f"{o}: {schedule}")
    
    # Counter Manning
    counter_manning = {c: [] for c in counters}
    for s in slots:
        manning = {c: 0 for c in counters}
        for o in officers:
            val = solver.Value(officer_slot[o, s])
            if val != -1:
                manning[counters[val]] += 1
        for c in counters:
            counter_manning[c].append(manning[c])
    
    print("\n=== Counter Manning ===")
    for c in counters:
        print(f"{c}: {counter_manning[c]}")
else:
    print("No feasible solution found.")


In [ ]:
from ortools.sat.python import cp_model

# === Parameters ===
officers = ["A", "B", "C"]
counters = ["C1", "C2", "C3"]
slots = list(range(15))

# Officer availability (start_slot, end_slot)
sos_availability = {
    "A": (1, 12),
    "B": (0, 12),
    "C": (0, 12)
}

# Minimum total break slots per officer
min_total_breaks = {
    "A": 4,
    "B": 3,
    "C": 2
}

# Allowed break block lengths
break_lengths = [2, 3]

# Maximum consecutive working slots
max_working_slots = 8

from ortools.sat.python import cp_model

def run_cp_model(officers, counters, slots, sos_availability, min_total_breaks,
                 break_lengths, max_working_slots,
                 COUNTER_CHANGE_WEIGHT, BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT):

    model = cp_model.CpModel()

    # === Decision Variables ===
    officer_slot = {}
    for o in officers:
        for s in slots:
            officer_slot[o, s] = model.NewIntVar(-1, len(counters)-1, f"officer_{o}_slot_{s}")

    # === Objective components ===
    working_slots = []
    counter_change_penalties = []
    break_timing_penalties = []
    counter_coverage_penalties = []

    # === Constraints per officer ===
    is_break = {}
    for o in officers:
        is_break[o] = [model.NewBoolVar(f"break_{o}_{s}") for s in slots]

        for s in slots:
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_break[o][s])
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_break[o][s].Not())
        model.Add(sum(is_break[o]) >= min_total_breaks[o])

        # Break blocks with no consecutive merges
        block_starts = []
        s_idx = 0
        while s_idx < len(slots):
            for length in break_lengths:
                if s_idx + length <= len(slots):
                    block = [is_break[o][s_idx + i] for i in range(length)]
                    block_start = model.NewBoolVar(f"break_block_{o}_{s_idx}_len{length}")
                    block_starts.append((s_idx, length, block_start))
                    model.Add(sum(block) == length).OnlyEnforceIf(block_start)
                    model.Add(sum(block) != length).OnlyEnforceIf(block_start.Not())
                    if s_idx + length < len(slots):
                        model.Add(is_break[o][s_idx + length] == 0).OnlyEnforceIf(block_start)
            s_idx += 1
        block_lengths_list = [length for (_, length, _) in block_starts]
        block_vars_list = [var for (_, _, var) in block_starts]
        model.Add(sum([l * v for l, v in zip(block_lengths_list, block_vars_list)]) >= min_total_breaks[o])

        # Stay at same counter until break
        for s in range(1, len(slots)):
            prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
            model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
            model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())

            curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())

            both_working = model.NewBoolVar(f"{o}_both_working_{s}")
            model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
            model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())

            model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(both_working)

        # Maximum consecutive working slots (sliding window)
        for start in range(len(slots) - max_working_slots):
            window_breaks = [is_break[o][start + i] for i in range(max_working_slots + 1)]
            model.Add(sum(window_breaks) >= 1)

        # Working slots & break timing penalty
        start_avail, end_avail = sos_availability[o]
        for s in slots:
            is_working = model.NewBoolVar(f"working_{o}_{s}")
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_working)
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_working.Not())
            working_slots.append(is_working)

            # Break timing penalty (avoid first 4 or last 4 slots of availability)
            if s - start_avail < 4 or end_avail - s <= 4:
                break_penalty = model.NewBoolVar(f"break_penalty_{o}_{s}")
                model.Add(is_break[o][s] == 1).OnlyEnforceIf(break_penalty)
                model.Add(is_break[o][s] == 0).OnlyEnforceIf(break_penalty.Not())
                break_timing_penalties.append(break_penalty)

        # Counter change penalties
        for s in range(1, len(slots)):
            changed = model.NewBoolVar(f"{o}_changed_{s}")
            model.Add(officer_slot[o, s] != officer_slot[o, s-1]).OnlyEnforceIf(changed)
            model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(changed.Not())

            prev_working = model.NewBoolVar(f"{o}_prev_working_change_{s}")
            model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
            model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())

            curr_working = model.NewBoolVar(f"{o}_curr_working_change_{s}")
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())

            both_working = model.NewBoolVar(f"{o}_both_working_change2_{s}")
            model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
            model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())

            penalty_if_working = model.NewBoolVar(f"penalty_if_working_{o}_{s}")
            model.AddBoolAnd([changed, both_working]).OnlyEnforceIf(penalty_if_working)
            model.AddBoolOr([changed.Not(), both_working.Not()]).OnlyEnforceIf(penalty_if_working.Not())
            counter_change_penalties.append(penalty_if_working)

    # Max 1 officer per counter per slot
    for c_index, c in enumerate(counters):
        for s in slots:
            working_at_counter = []
            for o in officers:
                is_at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(is_at_c)
                model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(is_at_c.Not())
                working_at_counter.append(is_at_c)
            model.Add(sum(working_at_counter) <= 1)

    # Counter coverage penalty: <2 counters open
    for s in slots:
        counter_open = []
        for c_index, c in enumerate(counters):
            officer_at_c = []
            for o in officers:
                at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(at_c)
                model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(at_c.Not())
                officer_at_c.append(at_c)
            has_officer = model.NewBoolVar(f"{c}_open_{s}")
            model.AddBoolOr(officer_at_c).OnlyEnforceIf(has_officer)
            model.AddBoolAnd([x.Not() for x in officer_at_c]).OnlyEnforceIf(has_officer.Not())
            counter_open.append(has_officer)
        num_open = model.NewIntVar(0, len(counters), f"num_open_{s}")
        model.Add(num_open == sum(counter_open))
        penalty = model.NewBoolVar(f"coverage_penalty_{s}")
        model.Add(num_open < 2).OnlyEnforceIf(penalty)
        model.Add(num_open >= 2).OnlyEnforceIf(penalty.Not())
        counter_coverage_penalties.append(penalty)

    # === Objective ===
    model.Maximize(
        sum(working_slots)
        - COUNTER_CHANGE_WEIGHT * sum(counter_change_penalties)
        - BREAK_TIME_WEIGHT * sum(break_timing_penalties)
        - MIN_COUNTER_WEIGHT * sum(counter_coverage_penalties)
    )

    # === Solve ===
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    return solver, status, officer_slot

def print_model_output(solver, status):
    # === Print schedules and counter manning ===
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("=== Officer Schedules ===")
        for o in officers:
            schedule = []
            for s in slots:
                val = solver.Value(officer_slot[o, s])
                schedule.append('#' if val == -1 else counters[val])
            print(f"{o}: {schedule}")
        
        # Counter Manning
        counter_manning = {c: [] for c in counters}
        for s in slots:
            manning = {c: 0 for c in counters}
            for o in officers:
                val = solver.Value(officer_slot[o, s])
                if val != -1:
                    manning[counters[val]] += 1
            for c in counters:
                counter_manning[c].append(manning[c])
        
        print("\n=== Counter Manning ===")
        for c in counters:
            print(f"{c}: {counter_manning[c]}")
    else:
        print("No feasible solution found.")


In [ ]:
officers, counters, availability, break_requirements = build_model_inputs(input_avail)
print(break_requirements)

In [101]:
print(availability)

{'O1': [0, 1, 2, 3, 4, 5, 6, 7], 'O2': [40, 41, 42, 43, 44, 45, 46], 'O3': [12, 13, 14, 15, 16, 17, 42, 43, 44, 45, 46], 'O4': [12, 13, 14, 15, 16, 17, 42, 43, 44, 45, 46], 'O5': [0, 1, 2, 3, 4, 5, 12, 13, 14, 15, 16, 17, 42, 43, 44, 45, 46], 'O6': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], 'O7': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], 'O8': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], 'O9': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], 'O10': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], 'O11': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], 'O12': [2, 3, 4, 5, 6, 

In [104]:
from ortools.sat.python import cp_model

def run_cp_model(
    officers, counters, slots, sos_availability, min_total_breaks, 
    break_lengths, max_working_slots,
    COUNTER_CHANGE_WEIGHT, BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT
):
    model = cp_model.CpModel()

    # === Decision Variables ===
    officer_slot = {}
    for o in officers:
        for s in slots:
            officer_slot[o, s] = model.NewIntVar(-1, len(counters)-1, f"officer_{o}_slot_{s}")

    # === Objective components ===
    working_slots = []
    counter_change_penalties = []
    break_timing_penalties = []
    counter_coverage_penalties = []

    # === Constraints per officer ===
    is_break = {}
    for o in officers:
        is_break[o] = [model.NewBoolVar(f"break_{o}_{s}") for s in slots]

        # Force unavailable slots to be breaks
        avail_slots = set(sos_availability[o])
        for s in slots:
            if s not in avail_slots:
                model.Add(officer_slot[o, s] == -1)
                model.Add(is_break[o][s] == 1)
            else:
                model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_break[o][s])
                model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_break[o][s].Not())

        # Total breaks
        model.Add(sum(is_break[o]) >= min_total_breaks[o])

        # Break blocks with no consecutive merges
        block_starts = []
        s_idx = 0
        while s_idx < len(slots):
            for length in break_lengths:
                if s_idx + length <= len(slots):
                    block = [is_break[o][s_idx + i] for i in range(length)]
                    block_start = model.NewBoolVar(f"break_block_{o}_{s_idx}_len{length}")
                    block_starts.append((length, block_start))
                    model.Add(sum(block) == length).OnlyEnforceIf(block_start)
                    model.Add(sum(block) != length).OnlyEnforceIf(block_start.Not())
                    if s_idx + length < len(slots):
                        model.Add(is_break[o][s_idx + length] == 0).OnlyEnforceIf(block_start)
            s_idx += 1
        block_lengths_list = [l for l, _ in block_starts]
        block_vars_list = [v for _, v in block_starts]
        model.Add(sum([l * v for l, v in zip(block_lengths_list, block_vars_list)]) >= min_total_breaks[o])

        # Stay at same counter until break
        for s in range(1, len(slots)):
            prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
            model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
            model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())

            curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())

            both_working = model.NewBoolVar(f"{o}_both_working_{s}")
            model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
            model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())

            model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(both_working)

        # Maximum consecutive working slots (sliding window)
        for start in range(len(slots) - max_working_slots):
            window_breaks = [is_break[o][start + i] for i in range(max_working_slots + 1)]
            model.Add(sum(window_breaks) >= 1)

        # Working slots & break timing penalty
        if avail_slots:
            first_slot = min(avail_slots)
            last_slot = max(avail_slots)
            for s in slots:
                is_working = model.NewBoolVar(f"working_{o}_{s}")
                model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_working)
                model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_working.Not())
                working_slots.append(is_working)

                # Only penalize breaks if s is available
                if s in avail_slots and (s - first_slot < 4 or last_slot - s < 4):
                    break_penalty = model.NewBoolVar(f"break_penalty_{o}_{s}")
                    model.Add(is_break[o][s] == 1).OnlyEnforceIf(break_penalty)
                    model.Add(is_break[o][s] == 0).OnlyEnforceIf(break_penalty.Not())
                    break_timing_penalties.append(break_penalty)

        # Counter change penalties
        for s in range(1, len(slots)):
            changed = model.NewBoolVar(f"{o}_changed_{s}")
            model.Add(officer_slot[o, s] != officer_slot[o, s-1]).OnlyEnforceIf(changed)
            model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(changed.Not())

            prev_working = model.NewBoolVar(f"{o}_prev_working_change_{s}")
            model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
            model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())

            curr_working = model.NewBoolVar(f"{o}_curr_working_change_{s}")
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())

            both_working = model.NewBoolVar(f"{o}_both_working_change2_{s}")
            model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
            model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())

            penalty_if_working = model.NewBoolVar(f"penalty_if_working_{o}_{s}")
            model.AddBoolAnd([changed, both_working]).OnlyEnforceIf(penalty_if_working)
            model.AddBoolOr([changed.Not(), both_working.Not()]).OnlyEnforceIf(penalty_if_working.Not())
            counter_change_penalties.append(penalty_if_working)

    # Max 1 officer per counter per slot
    for c_index, c in enumerate(counters):
        for s in slots:
            working_at_counter = []
            for o in officers:
                is_at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(is_at_c)
                model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(is_at_c.Not())
                working_at_counter.append(is_at_c)
            model.Add(sum(working_at_counter) <= 1)

    # Counter coverage penalty: <2 counters open
    for s in slots:
        counter_open = []
        for c_index, c in enumerate(counters):
            officer_at_c = []
            for o in officers:
                at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(at_c)
                model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(at_c.Not())
                officer_at_c.append(at_c)
            has_officer = model.NewBoolVar(f"{c}_open_{s}")
            model.AddBoolOr(officer_at_c).OnlyEnforceIf(has_officer)
            model.AddBoolAnd([x.Not() for x in officer_at_c]).OnlyEnforceIf(has_officer.Not())
            counter_open.append(has_officer)
        num_open = model.NewIntVar(0, len(counters), f"num_open_{s}")
        model.Add(num_open == sum(counter_open))
        penalty = model.NewBoolVar(f"coverage_penalty_{s}")
        model.Add(num_open < 2).OnlyEnforceIf(penalty)
        model.Add(num_open >= 2).OnlyEnforceIf(penalty.Not())
        counter_coverage_penalties.append(penalty)

    # === Objective ===
    model.Maximize(
        sum(working_slots)
        - COUNTER_CHANGE_WEIGHT * sum(counter_change_penalties)
        - BREAK_TIME_WEIGHT * sum(break_timing_penalties)
        - MIN_COUNTER_WEIGHT * sum(counter_coverage_penalties)
    )

    # === Solve ===
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    return solver, status, officer_slot


In [114]:
COUNTER_CHANGE_WEIGHT,BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT = 0.5,2,2.5
officers, counters, availability, break_requirements = build_model_inputs(input_avail, 20)
min_total_breaks = break_requirements
slots = list(range(48))
max_working_slots = 10
break_lengths = [1,2,3]


# === Parameters ===
officers = ["A", "B", "C"]
counters = ["C1", "C2", "C3"]
slots = list(range(15))

# Officer availability (start_slot, end_slot)
availability = {
    "A": (1, 12),
    "B": (0, 12),
    "C": (0, 12)
}

# Minimum total break slots per officer
min_total_breaks = {
    "A": 4,
    "B": 3,
    "C": 2
}

# Allowed break block lengths
break_lengths = [2, 3]

# Maximum consecutive working slots
max_working_slots = 8


print(officers)
print(counters)
print(slots)
print(availability)
print(min_total_breaks)
print(break_lengths)
print(max_working_slots)
solver, status, officer_slot = run_cp_model(officers, counters, slots, availability, min_total_breaks, break_lengths, max_working_slots, COUNTER_CHANGE_WEIGHT,BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT)
print_model_output(solver, status)

['A', 'B', 'C']
['C1', 'C2', 'C3']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'A': (1, 12), 'B': (0, 12), 'C': (0, 12)}
{'A': 4, 'B': 3, 'C': 2}
[2, 3]
8
No feasible solution found.


In [116]:
#trying to rectify no feasible solution due to sliding window checking for breaks outside sos interval
from ortools.sat.python import cp_model

def run_cp_model(officers, counters, slots, sos_availability, min_total_breaks, 
                 break_lengths, max_working_slots,
                 COUNTER_CHANGE_WEIGHT, BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT):

    model = cp_model.CpModel()

    # === Decision Variables ===
    officer_slot = {}
    for o in officers:
        for s in slots:
            officer_slot[o, s] = model.NewIntVar(-1, len(counters)-1, f"officer_{o}_slot_{s}")

    # === Objective components ===
    working_slots = []
    counter_change_penalties = []
    break_timing_penalties = []
    counter_coverage_penalties = []

    # === Constraints per officer ===
    is_break = {}
    for o in officers:
        is_break[o] = [model.NewBoolVar(f"break_{o}_{s}") for s in slots]

        # Officer can only work in their available slots
        available_slots = set()
        for interval in sos_availability[o]:  # interval = (start, end)
            available_slots.update(range(interval[0], interval[1]+1))
        for s in slots:
            if s not in available_slots:
                model.Add(officer_slot[o, s] == -1)
                model.Add(is_break[o][s] == 1)

        # Minimum total breaks
        model.Add(sum(is_break[o]) >= min_total_breaks[o])

        # Break blocks with no consecutive merges
        block_starts = []
        s_idx = 0
        while s_idx < len(slots):
            for length in break_lengths:
                if s_idx + length <= len(slots):
                    block = [is_break[o][s_idx + i] for i in range(length)]
                    block_start = model.NewBoolVar(f"break_block_{o}_{s_idx}_len{length}")
                    block_starts.append((s_idx, length, block_start))
                    model.Add(sum(block) == length).OnlyEnforceIf(block_start)
                    model.Add(sum(block) != length).OnlyEnforceIf(block_start.Not())
                    if s_idx + length < len(slots):
                        model.Add(is_break[o][s_idx + length] == 0).OnlyEnforceIf(block_start)
            s_idx += 1
        block_lengths_list = [length for (_, length, _) in block_starts]
        block_vars_list = [var for (_, _, var) in block_starts]
        model.Add(sum([l * v for l, v in zip(block_lengths_list, block_vars_list)]) >= min_total_breaks[o])

        # Stay at same counter until break
        for s in range(1, len(slots)):
            prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
            model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
            model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())

            curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())

            both_working = model.NewBoolVar(f"{o}_both_working_{s}")
            model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
            model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())

            model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(both_working)

        # Maximum consecutive working slots (sliding window) **within each SOS interval**
        for interval in sos_availability[o]:
            start, end = interval
            for s in range(start, end - max_working_slots):
                window_breaks = [is_break[o][s + i] for i in range(max_working_slots + 1)]
                model.Add(sum(window_breaks) >= 1)

        # Working slots & break timing penalty
        for interval in sos_availability[o]:
            start_avail, end_avail = interval
            for s in range(start_avail, end_avail + 1):
                is_working = model.NewBoolVar(f"working_{o}_{s}")
                model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_working)
                model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_working.Not())
                working_slots.append(is_working)

                # Break timing penalty (avoid first 4 or last 4 slots of availability)
                if s - start_avail < 4 or end_avail - s < 4:
                    break_penalty = model.NewBoolVar(f"break_penalty_{o}_{s}")
                    model.Add(is_break[o][s] == 1).OnlyEnforceIf(break_penalty)
                    model.Add(is_break[o][s] == 0).OnlyEnforceIf(break_penalty.Not())
                    break_timing_penalties.append(break_penalty)

        # Counter change penalties
        for s in range(1, len(slots)):
            changed = model.NewBoolVar(f"{o}_changed_{s}")
            model.Add(officer_slot[o, s] != officer_slot[o, s-1]).OnlyEnforceIf(changed)
            model.Add(officer_slot[o, s] == officer_slot[o, s-1]).OnlyEnforceIf(changed.Not())

            prev_working = model.NewBoolVar(f"{o}_prev_working_change_{s}")
            model.Add(officer_slot[o, s-1] != -1).OnlyEnforceIf(prev_working)
            model.Add(officer_slot[o, s-1] == -1).OnlyEnforceIf(prev_working.Not())

            curr_working = model.NewBoolVar(f"{o}_curr_working_change_{s}")
            model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(curr_working)
            model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(curr_working.Not())

            both_working = model.NewBoolVar(f"{o}_both_working_change2_{s}")
            model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
            model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())

            penalty_if_working = model.NewBoolVar(f"penalty_if_working_{o}_{s}")
            model.AddBoolAnd([changed, both_working]).OnlyEnforceIf(penalty_if_working)
            model.AddBoolOr([changed.Not(), both_working.Not()]).OnlyEnforceIf(penalty_if_working.Not())
            counter_change_penalties.append(penalty_if_working)

    # Max 1 officer per counter per slot
    for c_index, c in enumerate(counters):
        for s in slots:
            working_at_counter = []
            for o in officers:
                is_at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(is_at_c)
                model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(is_at_c.Not())
                working_at_counter.append(is_at_c)
            model.Add(sum(working_at_counter) <= 1)

    # Counter coverage penalty: <2 counters open
    for s in slots:
        counter_open = []
        for c_index, c in enumerate(counters):
            officer_at_c = []
            for o in officers:
                at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(at_c)
                model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(at_c.Not())
                officer_at_c.append(at_c)
            has_officer = model.NewBoolVar(f"{c}_open_{s}")
            model.AddBoolOr(officer_at_c).OnlyEnforceIf(has_officer)
            model.AddBoolAnd([x.Not() for x in officer_at_c]).OnlyEnforceIf(has_officer.Not())
            counter_open.append(has_officer)
        num_open = model.NewIntVar(0, len(counters), f"num_open_{s}")
        model.Add(num_open == sum(counter_open))
        penalty = model.NewBoolVar(f"coverage_penalty_{s}")
        model.Add(num_open < 2).OnlyEnforceIf(penalty)
        model.Add(num_open >= 2).OnlyEnforceIf(penalty.Not())
        counter_coverage_penalties.append(penalty)

    # === Objective ===
    model.Maximize(
        sum(working_slots)
        - COUNTER_CHANGE_WEIGHT * sum(counter_change_penalties)
        - BREAK_TIME_WEIGHT * sum(break_timing_penalties)
        - MIN_COUNTER_WEIGHT * sum(counter_coverage_penalties)
    )

    # === Solve ===
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    return solver, status, officer_slot


In [119]:
print(officers)
print(counters)
print(slots)
print(sos_availability)
print(min_total_breaks)
print(break_lengths)
print(max_working_slots)
solver, status, officer_slot = run_cp_model(officers, counters, slots, sos_availability, min_total_breaks, break_lengths, max_working_slots, COUNTER_CHANGE_WEIGHT,BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT)
print_model_output(solver, status)


['A', 'B', 'C']
['C1', 'C2', 'C3']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'A': (1, 12), 'B': (0, 12), 'C': (0, 12)}
{'A': 4, 'B': 3, 'C': 2}
[2, 3]
8


TypeError: 'int' object is not subscriptable

In [2]:
from ortools.sat.python import cp_model

slot_length = 15
def hhmm_to_slot(hhmm: str, slot_length=slot_length, NUM_SLOTS = 48):
    """Convert 'HHMM' string to slot index. Default slot = 30 min."""
    t = int(hhmm)
    h = t // 100
    m = t % 100
    slot = (h - 10) * 4 + (m // slot_length)
    return max(0, min(NUM_SLOTS - 1, slot))

def calculate_total_break2(officer_intervals):
    officers_break_quota = {}
    for officer in officer_intervals:
        total_break = 0
        for each_interval in officer_intervals[officer]:
            s, e = each_interval
            if e - s >= 36:   # >= 9 hours
                total_break = 8
                break
            elif e - s >= 20: # >= 5 hours
                total_break = max(total_break, 3)
            elif e - s >= 10: # >= 2.5 hours
                total_break = max(total_break, 2)
        officers_break_quota[officer] = total_break
    return officers_break_quota

def build_model_inputs(input_avail: list[str], slot_length=slot_length):
    officers = [f"O{i+1}" for i in range(len(input_avail))]
    counters = [f"C{i+1}" for i in range(40)]  # 40 counters
    
    sos_availability = {}
    availability = {}
    for i, avail_str in enumerate(input_avail):
        intervals = [s.strip() for s in avail_str.split(",") if s.strip()]
        officer_intervals = []
        for interval in intervals:
            start_str, end_str = interval.split("-")
            start_slot = hhmm_to_slot(start_str, slot_length)
            end_slot = hhmm_to_slot(end_str, slot_length)
            officer_intervals.append((start_slot, end_slot))
        sos_availability[officers[i]] = officer_intervals
        availability[officers[i]] = officer_intervals
    
    break_requirements = calculate_total_break2(sos_availability)
    return officers, counters, availability, break_requirements

def run_cp_model(officers, counters, slots, sos_availability, min_total_breaks, 
                 break_lengths, max_working_slots,
                 COUNTER_CHANGE_WEIGHT, BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT):

    model = cp_model.CpModel()
    officer_slot = {}
    is_break = {}

    # === Create variables ===
    for o in officers:
        is_break[o] = []
        for s in slots:
            # Check if slot is inside SOS interval
            in_sos = any(start <= s < end for start, end in sos_availability[o])
            if not in_sos:
                # Outside SOS: fixed value -2
                officer_slot[o, s] = -2
                is_break[o].append(None)
            else:
                # Inside SOS: decision variable
                officer_slot[o, s] = model.NewIntVar(-1, len(counters)-1, f"officer_{o}_slot_{s}")
                b = model.NewBoolVar(f"break_{o}_{s}")
                is_break[o].append(b)
                model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(b)
                model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(b.Not())

    # === Working slots & penalties ===
    working_slots = []
    break_timing_penalties = []
    counter_change_penalties = []
    counter_coverage_penalties = []

    for o in officers:
        # Minimum total breaks
        model.Add(sum([b for b in is_break[o] if b is not None]) >= min_total_breaks[o])

        # Maximum consecutive working slots per SOS interval
        for start_s, end_s in sos_availability[o]:
            for s_start in range(start_s, end_s - max_working_slots):
                window_breaks = [is_break[o][s_start + i] for i in range(max_working_slots + 1)]
                model.Add(sum(window_breaks) >= 1)

        # Working slots & break timing penalties
        for start_s, end_s in sos_availability[o]:
            for s in range(start_s, end_s):
                is_working = model.NewBoolVar(f"working_{o}_{s}")
                model.Add(officer_slot[o, s] != -1).OnlyEnforceIf(is_working)
                model.Add(officer_slot[o, s] == -1).OnlyEnforceIf(is_working.Not())
                working_slots.append(is_working)

                # Optional break timing near interval edges
                if s - start_s < 4 or end_s - s <= 4:
                    break_penalty = model.NewBoolVar(f"break_penalty_{o}_{s}")
                    model.Add(is_break[o][s] == 1).OnlyEnforceIf(break_penalty)
                    model.Add(is_break[o][s] == 0).OnlyEnforceIf(break_penalty.Not())
                    break_timing_penalties.append(break_penalty)

        # Counter change penalties
        for s in range(1, len(slots)):
            # Skip if any slot is outside SOS
            prev_slot = officer_slot[o, s-1]
            curr_slot = officer_slot[o, s]
            if (isinstance(prev_slot, int) and prev_slot == -2) or (isinstance(curr_slot, int) and curr_slot == -2):
                continue

            changed = model.NewBoolVar(f"{o}_changed_{s}")
            model.Add(curr_slot != prev_slot).OnlyEnforceIf(changed)
            model.Add(curr_slot == prev_slot).OnlyEnforceIf(changed.Not())

            prev_working = model.NewBoolVar(f"{o}_prev_working_{s}")
            curr_working = model.NewBoolVar(f"{o}_curr_working_{s}")
            model.Add(prev_slot != -1).OnlyEnforceIf(prev_working)
            model.Add(prev_slot == -1).OnlyEnforceIf(prev_working.Not())
            model.Add(curr_slot != -1).OnlyEnforceIf(curr_working)
            model.Add(curr_slot == -1).OnlyEnforceIf(curr_working.Not())

            both_working = model.NewBoolVar(f"{o}_both_working_{s}")
            model.AddBoolAnd([prev_working, curr_working]).OnlyEnforceIf(both_working)
            model.AddBoolOr([prev_working.Not(), curr_working.Not()]).OnlyEnforceIf(both_working.Not())

            penalty_if_working = model.NewBoolVar(f"penalty_if_working_{o}_{s}")
            model.AddBoolAnd([changed, both_working]).OnlyEnforceIf(penalty_if_working)
            model.AddBoolOr([changed.Not(), both_working.Not()]).OnlyEnforceIf(penalty_if_working.Not())
            counter_change_penalties.append(penalty_if_working)

    # === Max 1 officer per counter per slot ===
    for c_index, c in enumerate(counters):
        for s in slots:
            working_at_counter = []
            for o in officers:
                val = officer_slot[o, s]
                if isinstance(val, int) and val == -2:
                    continue
                is_at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(officer_slot[o, s] == c_index).OnlyEnforceIf(is_at_c)
                model.Add(officer_slot[o, s] != c_index).OnlyEnforceIf(is_at_c.Not())
                working_at_counter.append(is_at_c)
            if working_at_counter:
                model.Add(sum(working_at_counter) <= 1)

    # === Counter coverage penalties ===
    for s in slots:
        counter_open = []
        for c_index, c in enumerate(counters):
            officer_at_c = []
            for o in officers:
                val = officer_slot[o, s]
                if isinstance(val, int) and val == -2:
                    continue
                at_c = model.NewBoolVar(f"{o}_at_{c}_{s}")
                model.Add(val == c_index).OnlyEnforceIf(at_c)
                model.Add(val != c_index).OnlyEnforceIf(at_c.Not())
                officer_at_c.append(at_c)
            if officer_at_c:
                has_officer = model.NewBoolVar(f"{c}_open_{s}")
                model.AddBoolOr(officer_at_c).OnlyEnforceIf(has_officer)
                model.AddBoolAnd([x.Not() for x in officer_at_c]).OnlyEnforceIf(has_officer.Not())
                counter_open.append(has_officer)
        if counter_open:
            num_open = model.NewIntVar(0, len(counters), f"num_open_{s}")
            model.Add(num_open == sum(counter_open))
            penalty = model.NewBoolVar(f"coverage_penalty_{s}")
            model.Add(num_open < 2).OnlyEnforceIf(penalty)
            model.Add(num_open >= 2).OnlyEnforceIf(penalty.Not())
            counter_coverage_penalties.append(penalty)

    # === Objective ===
    model.Maximize(
        sum(working_slots)
        - COUNTER_CHANGE_WEIGHT * sum(counter_change_penalties)
        - BREAK_TIME_WEIGHT * sum(break_timing_penalties)
        - MIN_COUNTER_WEIGHT * sum(counter_coverage_penalties)
    )

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 60 
    status = solver.Solve(model)
    return solver, status, officer_slot


def print_model_output(solver, status, officers, counters, slots, officer_slot):
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("=== Officer Schedules ===")
        print("Legend: # = break, X = outside SOS, Cn = counter number\n")

        for o in officers:
            schedule = []
            for s in slots:
                val = solver.Value(officer_slot[o, s])
                if val == -1:
                    schedule.append('#')   # break inside SOS
                elif val == -2:
                    schedule.append('X')   # outside SOS
                else:
                    schedule.append(counters[val])
            print(f"{o}: {schedule}")

        print("\n=== Counter Manning ===")
        counter_manning = {c: [] for c in counters}
        for s in slots:
            manning = {c: 0 for c in counters}
            for o in officers:
                val = solver.Value(officer_slot[o, s])
                if val != -1 and val != -2:
                    manning[counters[val]] += 1
            for c in counters:
                counter_manning[c].append(manning[c])
        for c in counters:
            print(f"{c}: {counter_manning[c]}")
    else:
        print("No feasible solution found.")


# === Example Usage ===

input_avail = [
    '1000-1200','2000-2200','1300-1430,2030-2200','1300-1430,2030-2200',
    '1300-1430,2030-2200,1000-1130','1000-1600','1000-1600','1030-1900',
    '1030-1900','1030-1900','1030-1900','1030-1900','1100-2200','1100-2200',
    '1100-2200','1200-2200','1200-2200','1145-1830','1145-2200','1200-2200',
    '1145-2200','1145-2200','1230-1400','1130-1300','1300-1430','1230-1630',
    '1600-1830','1600-1830','1400-1830','1400-1830','1000-1200','2000-2200',
    '1800-2030','1700-2200'
]

COUNTER_CHANGE_WEIGHT, BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT = 0.5, 2, 2.5
slots = list(range(48))
break_lengths = [2, 3]
max_working_slots = 8

officers, counters, availability, break_requirements = build_model_inputs(input_avail, 15)
min_total_breaks = break_requirements

solver, status, officer_slot = run_cp_model(
    officers, counters, slots, availability, min_total_breaks,
    break_lengths, max_working_slots,
    COUNTER_CHANGE_WEIGHT, BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT
)

print_model_output(solver, status, officers, counters, slots, officer_slot)


=== Officer Schedules ===
Legend: # = break, X = outside SOS, Cn = counter number

O1: ['C1', 'C1', 'C6', 'C6', 'C6', 'C6', 'C6', 'C6', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
O2: ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'C16', 'C16', 'C16', 'C16', 'C16', 'C16', 'C16', 'X']
O3: ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'C14', 'C14', 'C14', 'C14', 'C14', 'X']
O4: ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'C23', 'C23', 'C23', 'C23', 'C23', 'C23', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X

In [121]:
input_avail = [
    '1000-1200','2000-2200','1300-1430,2030-2200','1300-1430,2030-2200',
    '1300-1430,2030-2200,1000-1130','1000-1600','1000-1600','1030-1900',
    '1030-1900','1030-1900','1030-1900','1030-1900','1100-2200','1100-2200',
    '1100-2200','1200-2200','1200-2200','1145-1830','1145-2200','1200-2200',
    '1145-2200','1145-2200','1230-1400','1130-1300','1300-1430','1230-1630',
    '1600-1830','1600-1830','1400-1830','1400-1830','1000-1200','2000-2200',
    '1800-2030','1700-2200'
]



COUNTER_CHANGE_WEIGHT,BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT = 0.5,2,2.5
slots = list(range(48))
# Allowed break block lengths
break_lengths = [2, 3, 4]
# Maximum consecutive working slots
max_working_slots = 12
officers, counters, availability, break_requirements = build_model_inputs(input_avail, 20)
min_total_breaks = break_requirements
print(officers)
print(counters)
print(slots)
print(availability)
print(min_total_breaks)
print(break_lengths)
print(max_working_slots)
solver, status, officer_slot = run_cp_model(officers, counters, slots, availability, min_total_breaks, break_lengths, max_working_slots, COUNTER_CHANGE_WEIGHT,BREAK_TIME_WEIGHT, MIN_COUNTER_WEIGHT)
print_model_output(solver, status)

['O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13', 'O14', 'O15', 'O16', 'O17', 'O18', 'O19', 'O20', 'O21', 'O22', 'O23', 'O24', 'O25', 'O26', 'O27', 'O28', 'O29', 'O30', 'O31', 'O32', 'O33', 'O34']
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
{'O1': [(0, 8)], 'O2': [(40, 47)], 'O3': [(12, 17), (41, 47)], 'O4': [(12, 17), (41, 47)], 'O5': [(12, 17), (41, 47), (0, 5)], 'O6': [(0, 24)], 'O7': [(0, 24)], 'O8': [(1, 36)], 'O9': [(1, 36)], 'O10': [(1, 36)], 'O11': [(1, 36)], 'O12': [(1, 36)], 'O13': [(4, 47)], 'O14': [(4, 47)], 'O15': [(4, 47)], 'O16': [(8, 

NotImplementedError: Evaluating a BoundedLinearExpression 'officer_O1_slot_0 == -2'instance as a Boolean is not supported.